In [ ]:
#Imports and preliminary settings

import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import keras.backend as K

plt.rc('font', size=16)

from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)




In [ ]:
#Calculate some statistics

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive
%ls

dataset = pd.read_csv("Training.csv")

print("Shape", dataset.shape)
print("\n\nMeans\n\n", dataset.mean())

print("\n\nStandard dev\n\n", dataset.std())

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

print("\n\nCovariance matrix\n\n", dataset.cov())

print("\n\nCorrelation matrix\n\n", dataset.corr())

In [ ]:
# Prepare the dataset

# INSPECT DATAFRAME
dataset.head()
dataset.info()
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
inspect_dataframe(dataset, dataset.columns)

val_size = 3800
X_train_raw = dataset.iloc[:-val_size]
# y_train_raw = y.iloc[:-val_size]
X_val_raw = dataset.iloc[-val_size:]
# y_test_raw = y.iloc[-val_size:]
print(X_train_raw.shape, X_val_raw.shape)

# Normalize both features and labels
X_min = X_train_raw.min()
X_max = X_train_raw.max()

X_train_raw = (X_train_raw-X_min)/(X_max-X_min)
X_val_raw = (X_val_raw-X_min)/(X_max-X_min)


window = 200
stride = 10

future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
print(future.shape)

def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels


target_labels = dataset.columns

# Set this to 864 if use direct forecasting or 1 if use autoregression
telescope = 864

X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_val_raw, target_labels, window, stride, telescope)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

inspect_multivariate(X_train, y_train, target_labels, telescope)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 200



In [ ]:
#This was the best model that we obtained using RNN. With a window=100
#it got a test RMSE of 3.67

input_shape=X_train.shape[1:]
output_shape=(864,7)
epochs=200
batch_size=64


def build_Rnn_model(input_shape, output_shape):
    
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    #A bunch of convolutions and pooling
    conv = tfkl.Conv1D(128,3,padding='same',activation='relu')(input_layer)
    pool = tfkl.MaxPool1D()(conv)
    
    drop = tfkl.Dropout(0.5)(pool)
    conv = tfkl.Conv1D(256,3,padding='same',activation='relu')(drop)

    pool = tfkl.MaxPool1D()(conv)
    drop = tfkl.Dropout(0.5)(pool)

    #Bidirectional RNN with 256 units
    lstm = tfkl.Bidirectional(tf.keras.layers.SimpleRNN(256,return_sequences=True))(drop)

    #One more convolution and pooling
    conv = tfkl.Conv1D(256,3,padding='same',activation='relu')(lstm)
    pool = tfkl.MaxPool1D()(conv)
    
    
    globavg = tfkl.GlobalAveragePooling1D()(conv)
    drop = tfkl.Dropout(0.5)(globavg)
    flat = tfkl.Flatten()(drop)

    #Fully connected layers
    d2 = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(flat)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(d2)

    model=tfk.Model(inputs=input_layer, outputs=output_layer, name='best_rnn')

    
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(0.00075), metrics=['mae', 'mse'])
    
    return model


mod = build_Rnn_model(input_shape, output_shape)
mod.summary()

mod.fit(x=X_train, y=y_train, validation_data=(X_test, y_test),epochs=epochs, batch_size=batch_size,callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ])



In [ ]:
#This was our best model using custom attention class and 
#GRU. It achieved a test RMSE of 3.56 with window=150

input_shape=X_train.shape[1:]
output_shape=(864,7)
epochs=200
batch_size=64

#Our attention class, later used in the network construction
class attention(tfkl.Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

def create_RNN_with_attention(input_shape, output_shape):
    input_layer=tfkl.Input(shape=input_shape)

    #A bunch of convolutions and MaxPooling
    l = tfkl.Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
    l = tfkl.MaxPooling1D()(l)
    
    l = tfkl.Conv1D(filters=128, kernel_size=3, activation='relu')(l)
    l = tfkl.MaxPooling1D()(l)
    
    l = tfkl.Conv1D(filters=256, kernel_size=3, activation='relu')(l)
    l = tfkl.MaxPooling1D()(l)
    l = tfkl.BatchNormalization()(l)

    #Bidirectional GRU
    recurrent = tfkl.Bidirectional(tfkl.GRU(512, return_sequences=True, activation='relu'))(l)

    #Here we inserted the attention layer
    attention_layer = attention()(recurrent)
    
    #Now two dense layers
    drop = tfkl.Dropout(0.4)(attention_layer)
    d2=tfkl.Dense(output_shape[-1]*output_shape[-2], trainable=True, activation='relu')(drop)
    drop = tfkl.Dropout(0.3)(d2)

    #Output
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(d2)

    model=tfk.Model(input_layer,output_layer, name="best_attention")

    model.compile(loss='mse', optimizer=tfk.optimizers.Adam(0.00075), metrics=['mae'])

    return model

mod = create_RNN_with_attention(input_shape, output_shape)
mod.summary()

mod.fit(x=X_train, y=y_train, validation_data=(X_test, y_test),epochs=epochs, batch_size=batch_size,callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_mae', mode='min', patience=25, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ])


In [ ]:
#Finally, our best model. The first quarter was predicted by mod1,
#whose code is now reported.

input_shape=X_train.shape[1:]
output_shape=(288,7) #Output shape is smaller because mod1 only predicts the 1t quarter
epochs=200
batch_size=64

#In order to comply with mod1 predicting only a subset of the future, we 
#had to modify the build_sequence functionw when training it.
#This version is the original one, the one used when we tried to create one
#custom model per quarter. This is why it returns labels_q1, labels_q2, labels_q3.
def build_sequences(df, target_labels=['pollution'], window=500, stride=20, telescope=864):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels_q1 = []
    labels_q2 = []
    labels_q3 = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels_q1.append(temp_label[idx+window:idx+window+telescope//3])
        labels_q2.append(temp_label[idx+window+telescope//3:idx+window+2*telescope//3])
        labels_q3.append(temp_label[idx+window+2*telescope//3:idx+window+telescope])

    dataset = np.array(dataset)
    labels_q1 = np.array(labels_q1)
    labels_q2 = np.array(labels_q2)
    labels_q3 = np.array(labels_q3)
    return dataset, labels_q1, labels_q2, labels_q3


target_labels=dataset.columns
val_ratio=0.1

telescope = 864

X_train, y_train_1, y_train_2, y_train_3 = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test_1, y_test_2, y_test_3 = build_sequences(X_val_raw, target_labels, window, stride, telescope)

#It is a simple convolutional model using LSTM. At first we tried creating
#3 of these models, training each to predict exactly one quarter. This still improved our results
#allowing us to achieve a 3.53 RMSE by using a window=100. Then, we tried to only keep the 1st of these models to predicts 
#the 1st quarter,while the remaining two were predicted by the attention model that is shown above. 
#This resulted in a 3.4 test RMSE.

def build_model1(input_shape, output_shape):
    
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv = tfkl.Conv1D(64, 13, padding='same', activation='relu')(input_layer)
    bn = tfkl.BatchNormalization()(conv)

    conv = tfkl.Conv1D(128, 3, padding='same', activation='relu')(bn)
    bn = tfkl.BatchNormalization()(conv)

    mp = tfkl.MaxPool1D()(bn)
    bn = tfkl.BatchNormalization()(mp)

    lstm = tfkl.Bidirectional(tfkl.LSTM(1024, return_sequences=True))(bn)
    bn = tfkl.BatchNormalization()(lstm)

    gap = tfkl.GlobalAveragePooling1D()(bn)
    dp = tfkl.Dropout(.3)(gap)

    dense = tfkl.Dense(output_shape[-1] * output_shape[-2], activation='relu')(dp)
    output_layer = tfkl.Reshape((output_shape[-2], output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model


mod1 = build_model1(input_shape, output_shape)
mod1.summary()

mod1.fit(x=X_train, y=y_train_1, validation_data=(X_test, y_test_1),epochs=epochs, batch_size=batch_size,callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_mae', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ])


#For completeness, we also show the model.py's code for this submission

import os
import tensorflow as tf
import pandas as pd
import numpy as np

class model:
    def __init__(self, path):
        #Model to predict the first quarter
        self.model1 = tf.keras.models.load_model(os.path.join(path, 'SubmissionModel1'))

        #Model to predict the second and third quarter
        self.model2 = tf.keras.models.load_model(os.path.join(path, 'SubmissionModel'))

    def predict(self, X):
        
        # Insert your preprocessing here
        X=X.numpy()
        X_min = X.min(axis=0)
        X_max = X.max(axis=0)
        
        future = X[-100:]
        future = (future - X_min) / (X_max - X_min)
        future = np.expand_dims(future, axis=0)
        
        future2 = X[-100:]
        future2 = (future2 - X_min) / (X_max - X_min)
        future2 = np.expand_dims(future2, axis=0)

        #Predict the 1st quarter
        out1 = self.model1.predict(future)

        #Predict the rest
        out2 = self.model2.predict(future2)

        # Insert your postprocessing here

        out1 = out1 * (X_max - X_min) + X_min  # denormalize
        #Reshape
        out1 = np.reshape(out1, (288, 7))
        
        out2 = out2 * (X_max - X_min) + X_min  # denormalize
        #Reshape
        out2 = np.reshape(out2, (288*3, 7))
        #out2 = tf.convert_to_tensor(out2)
        
        #Discard the first quarter predicted by model2, i.e. the first 288 values.
        out2 = out2[288:288*3]
        
        #Concatenate to achieve the full prediction
        out = np.concatenate((out1,out2), axis=0)
        out = tf.convert_to_tensor(out)

        return out